In [1]:
import os
import glob

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import losses, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization

from sclouds.helpers import path_input
from sclouds.io.utils import (dataset_to_numpy, dataset_to_numpy_grid, 
                              dataset_to_numpy_order, get_xarray_dataset_for_period)

In [2]:
help(get_xarray_dataset_for_period)

Help on function get_xarray_dataset_for_period in module sclouds.io.utils:

get_xarray_dataset_for_period(start='2012-01-01', stop='2012-01-31')
    Reads data from the requested period into a xarray dataset.
    
    Parameteres
    ----------------------
    start : str
        Start of period. First day included. (default '2012-01-01')
    
    stop : str
        end of period. Last day included. (default '2012-01-31')
    
    Returns
    -----------------------
    data : xr.Dataset
        Dataset including all variables in the requested period.



In [3]:
files =  ['/home/hanna/lagrings/ERA5_monthly/2012_01_q.nc',
          '/home/hanna/lagrings/ERA5_monthly/2012_01_r.nc',
          '/home/hanna/lagrings/ERA5_monthly/2012_01_t2m.nc',
          '/home/hanna/lagrings/ERA5_monthly/2012_01_sp.nc',
          '/home/hanna/lagrings/ERA5_monthly/2012_01_tcc.nc']

In [4]:
data = get_xarray_dataset_for_period(start = "2012-01-01", stop = "2012-01-31")


Num files 5


In [5]:
X, y = dataset_to_numpy_grid(data, bias = False)

In [6]:
X.shape, y.shape

((744, 81, 161, 4), (744, 81, 161, 1))

In [7]:
# reshape to sequence length

In [8]:
#744/4

In [9]:
v = X.reshape(186, 4, 81, 161, 4)
w = y.reshape(186, 4, 81, 161, 1)

In [10]:
filters     = [256, 218, 64, 32]
kernel_size = [3, 3, 3, 3]

In [11]:
#from keras.layers import keras.layers.

seq =  Sequential()



In [12]:
LAT = (30,50)
LON = (-15,25)
SPATIAL_RESOLUTION = 0.25

In [13]:
import numpy as np

In [14]:
def train_test_split(data, test_split = 0.2):
    assert test_split < 1, 'test split is given as a decimal number, choose a number between 0, 1'
    indx = int(len(data)*(1-test_split))
    # returns train test
    return data[:indx], data[indx:]

In [15]:
def batch_normalize(data, seq_len = 4): 
    samples, metvars, lats, lons = data.shape
    #.mean(axis = 0).mean(axis=1).mean(axis=1)
    
    normalized = np.zeros((samples, metvars, lats, lons))
    means   = np.zeros(metvars)
    storage = np.zeros(metvars)
    
    for i in range(metvars):
        raveled = data[:, i, :, :].reshape(-1)
        m = raveled.mean()
        s = raveled.std()
        normalized[:, i, :, :] =  (data[:, i, :, :] - m)/s
        
    samples, metvars, lats, lons = normalized.shape
    assert seq_len % 4 == 0
    
    new_samples = int(samples/seq_len)
    normalized  = normalized.reshape( (new_samples, seq_len, metvars, lats, lons ) )
    
    return normalized, means, storage

## Normalize

In [24]:
from tensorflow.keras.layers import BatchNormalization

In [25]:


#from keras.layers import keras.layers.

seq =  Sequential()

# Begin with 2D convolutional LSTM layer
seq.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3),
                   input_shape = (4, 81, 161, 4),
                   padding = 'same', return_sequences=True, data_format='channels_last')) 

seq.add(keras.layers.ConvLSTM2D(filters=8, kernel_size=(3, 3),
                                  #input_shape=(None, 35, 60, 1),
                                  padding='same', return_sequences=True,  data_format='channels_last')) 

# Add 3x hidden 2D convolutions LSTM layers    
# Begin with 2D convolutional LSTM layer
seq.add(keras.layers.ConvLSTM2D(filters=1, kernel_size=(1, 1), # (1, 1)
                                #input_shape=(None, 40, 40, 1),
                                padding='same', return_sequences=True,  data_format='channels_last')) 


In [26]:
import datetime

In [27]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
seq.compile(loss='mean_squared_error', 
            optimizer=sgd)

log_dir = "/home/hanna/MS/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
#%tensorboard --logdir '/home/hanna/MS/logs/fit'

In [ ]:
history = seq.fit(x=v, y=w, batch_size=32, epochs=10, verbose=1, callbacks=None, 
                  validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, 
                  sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, 
                  validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)


Train on 186 samples
Epoch 1/10


In [ ]:
def plot(history, val = True):
    """ Plot history in """
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    if val:
        plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    return

In [ ]:
plot(history, val = False)

In [ ]:
#from keras.utils import plot_model
#plot_model(model, to_file=save_dir+'model.png')

In [ ]:
# Prepare model for training
model1.compile(
      loss = "mse",
      metrics = ["mae"],
      optimizer = "adam"
)
import datetime
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


history = model1.fit(x,
                    y,
                    batch_size = 32,
                    epochs = 4,
                    validation_split = 0.2,
                    callbacks=[tensorboard_callback]
                    )

plot(history)

In [5]:
# %tensorboard --logdir logs/fit

In [ ]:
history

In [2]:
from tensorkeras.models import Sequential
from keras.layers import ConvLSTM2D
#from keras.layers.normalization import BatchNormalization

#from keras.layers import keras.layers.

def build_model(filters = [256, 128, 64], kernal = [5, 5, 3], input_shape = (4, 4, 35, 60)):
    model =  Sequential()

    # Begin with 2D convolutional LSTM layer
    for i, filter_ in enumerate( filters ):
        k_s = kernal[i]
        
        if i == 0:
            model.add(ConvLSTM2D(filters = filter_, kernel_size = (k_s, k_s),
                                 input_shape = input_shape,
                                 padding = 'same', return_sequences=True, data_format='channels_first')) 
        else:
            model.add(ConvLSTM2D(filters = filter_, kernel_size =  (k_s, k_s),
                                              #input_shape = (256, 35, 60, 1),
                                              padding='same', return_sequences=True,  data_format='channels_first')) 

    # Add 3x hidden 2D convolutions LSTM layers    
    # Begin with 2D convolutional LSTM layer
    # Blir dette rett da..?
    model.add(keras.layers.ConvLSTM2D(filters=1, kernel_size=(1, 1),
                                      #input_shape=(None, 40, 40, 1),
                                      padding='same', return_sequences=True,  data_format='channels_first')) 
    return model

ModuleNotFoundError: No module named 'keras'

In [1]:


model =  Sequential()

# Begin with 2D convolutional LSTM layer
model.add(ConvLSTM2D(filters = 256, kernel_size = (5, 5),
                     input_shape = (4, 4, 35, 60),
                     padding = 'same', return_sequences=True, data_format='channels_first')) 

model.add(keras.layers.ConvLSTM2D(filters = 128, kernel_size = (5, 5),
                                  #input_shape = (256, 35, 60, 1),
                                  padding='same', return_sequences=True,  data_format='channels_first')) 

model.add(BatchNormalization())

model.add(keras.layers.ConvLSTM2D(filters=64, kernel_size=(3, 3),
                                  #input_shape=(None, 35, 60, 1),
                                  padding='same', return_sequences=True,  data_format='channels_first')) 

model.add(BatchNormalization())

# Add 3x hidden 2D convolutions LSTM layers    
# Begin with 2D convolutional LSTM layer
model.add(keras.layers.ConvLSTM2D(filters=1, kernel_size=(3, 3),
                                  #input_shape=(None, 40, 40, 1),
                                  padding='same', return_sequences=True,  data_format='channels_first')) 

# Prepare model for training
model.compile(
      loss = "mse",
      metrics = ["mae"],
      optimizer = "adam"
)

history2 = model.fit(x,
                    y,
                    batch_size = 32,
                    epochs = 40,
                    validation_split = 0.2, 
                    callbacks=[tensorboard_callback]
                    )

ModuleNotFoundError: No module named 'keras'

In [ ]:
history2

In [ ]:
plot(history2)

In [ ]:
from keras.models import model_from_json


def save_json(model, m = "model.json"):    
    """ Storing a config / architecture of the model."""
    # serialize model to JSON
    model_json = model.to_json()
    with open(m, "w") as json_file:
        json_file.write(model_json)
    return

def load_model(m = 'model.json'):
    """ Loading a stored config/ architecture  to a model. """
    # load json and create model
    json_file = open(m, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    return loaded_model

In [ ]:
model_storage = '/home/hanna/MS/stored_models/'

In [ ]:
save_json( model, model_storage + "example_keras.json" )

In [ ]:
model = load_model(  model_storage + "example_keras.json" )

In [ ]:
model.save_weights(model_storage + "example_keras.h5")

In [ ]:
model.load_weights(model_storage + "example_keras.h5")

## Implement your own/ Custumize metrics  